In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#Create model class that inherits nn.Module

class Model(nn.Module):
    #input layer(4 features of the flower) -->
    #  hidden layer1 (H1) (some neurons)--> 
    # hidden layer2 (H2) --> 
    # output(3 classes of iris flower)
    def __init__(self,input_features =4, h1=8,h2=8,output_classes=3):
        self.fc1= nn.Linear(input_features,h1)
        self.fc2= nn.Linear(h1,h2)
        self.output= nn.Linear(h2,output_classes)
    
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.output(x)
        return x

    